In [1]:
import pandas as pd
import numpy as np
from automation_process_dataV2 import united_functions
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [2]:
df_rating = pd.read_parquet('../../data/raw/data_model/ratings_train05.parquet')#Carga de datos

In [3]:
# muestrear 2% de regitros
_, subset_rating = train_test_split(df_rating, test_size=0.02, random_state=111)

In [26]:
# Guardar muestra para aplicarla en la app final
subset_rating.to_parquet('../../data/raw/data_model/final_data_used/subset_splitV2.parquet')

In [4]:
subset_rating.reset_index(inplace=True, drop=True)

In [5]:
subset_rating.shape

(20001, 6)

In [5]:
df_processed, list_user_movies= united_functions(subset_rating)# Script de procesamiento de datos y retorno de titulo con id

INICIANDO PROCESAMIENTO DE DATOS...
- Eliminacion de nulos..
- Eliminando variables no necesarias...
- Extrayendo año del titulo...
- Almacenando en variable copia del df original...
- Extraemos los generos de cada pelicula...
- Eliminamos variables no necesarias...
- Codificamos los generos y normalizamos para su correcto formato...
FIN DE PROCESAMIENTO DE DATOS


In [6]:
df_processed.shape

(20000, 21)

In [28]:
# Guardar resutado de procesamiento para aplicarla en la app final
df_processed.to_csv('../../data/raw/data_model/final_data_used/process_splitV2.csv')
list_user_movies.to_csv('../../data/raw/data_model/final_data_used/list_user_moviesV2.csv')

In [7]:
# Diccionario de list_user_movies
dicc_user_movies = {'userId':list_user_movies['userId'], 'movieId':list_user_movies['movieId'], 'title':list_user_movies['title']}

Similitud del coseno (compara cada pelicula con todas las encontradas)

In [8]:
simil_cosine=cosine_similarity(df_processed)# calculo de similitus del coseno

In [9]:
# Rellenamos la diagonal con NaN y convertimos a dataframe
np.fill_diagonal(simil_cosine, np.nan)
df_similCosine = pd.DataFrame(simil_cosine)

In [10]:
df_similCosine

,0,1,2,3,4,5,6,7,8,9,...,19990,19991,19992,19993,19994,19995,19996,19997,19998,19999
0,NaN,0.842656,0.607109,0.288322,0.544108,0.481809,0.702519,0.590835,0.846874,0.495868,...,0.473396,0.280579,0.578960,0.682836,0.683817,0.833724,0.869675,0.841092,0.628432,0.713236
1,0.842656,NaN,0.733138,0.310207,0.644210,0.290645,0.822146,0.349792,0.990699,0.588129,...,0.511196,0.301185,0.337543,0.805703,0.812214,0.996599,0.717141,0.999171,0.400708,0.834055
2,0.607109,0.733138,NaN,0.171774,0.506069,0.155326,0.608460,0.193433,0.727803,0.428583,...,0.280538,0.167716,0.182017,0.982183,0.601077,0.725879,0.531549,0.740619,0.236906,0.608358
3,0.288322,0.310207,0.171774,NaN,0.171720,0.543137,0.265371,0.666060,0.325113,0.549422,...,0.625106,0.387393,0.433246,0.243444,0.508439,0.303785,0.449792,0.301365,0.456003,0.521820
4,0.544108,0.644210,0.506069,0.171720,NaN,0.150540,0.546823,0.193153,0.653586,0.588053,...,0.278314,0.168455,0.177832,0.538573,0.528492,0.630032,0.477776,0.651252,0.492964,0.545922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.833724,0.996599,0.725879,0.303785,0.630032,0.289773,0.809076,0.342789,0.976409,0.586725,...,0.502933,0.294090,0.335038,0.794992,0.808161,NaN,0.705524,0.993912,0.378652,0.823204
19996,0.869675,0.717141,0.531549,0.449792,0.477776,0.606225,0.603503,0.749620,0.725331,0.617692,...,0.377032,0.225132,0.739746,0.588434,0.584139,0.705524,NaN,0.718949,0.769106,0.609109
19997,0.841092,0.999171,0.740619,0.301365,0.651252,0.280611,0.824025,0.339740,0.991823,0.587109,...,0.495837,0.292892,0.326397,0.808679,0.812690,0.993912,0.718949,NaN,0.393962,0.834079
19998,0.628432,0.400708,0.236906,0.456003,0.492964,0.598480,0.363407,0.745773,0.441263,0.613919,...,0.400708,0.241325,0.730781,0.331772,0.321011,0.378652,0.769106,0.393962,NaN,0.374204


In [27]:
# Guardar matriz similitudes coseno
df_similCosine.to_csv('cosine_csv/df_similCosineV2.csv')

Logica que implementa la recomendacion en base a un usuario y pelicula vista

In [11]:
def comprobar_existencia(diccionario, movie_id, user_id):    
    for i in range(len(diccionario['userId'])):
        if diccionario['userId'].iloc[i] == user_id and diccionario['movieId'].iloc[i] == movie_id:
            return True
    return False

In [12]:
def obtener_index_movieId(diccionario, movie_id):
    for i in range(len(diccionario['movieId'])):
        if diccionario['movieId'].iloc[i]==movie_id:
            index = i
            break
    return index

In [13]:
def get_title_from_movieId(user_id ,diccionario):
    index = diccionario['userId'].loc[diccionario['userId'] == user_id].index
    # Obtén los títulos de las películas
    titles_saws = diccionario['title'].loc[index]
    return titles_saws

In [14]:
# Funcion de recomendacion para un usuario y pelicula
def get_recommendations(user_id, movie_id,dicc_user_movies=dicc_user_movies, simil_cosine=df_similCosine, threshold=0.7):
    #Comprobamos que el usuario para la pelicula dada existe
    comprobar_user_movie_exist=comprobar_existencia(dicc_user_movies, movie_id, user_id)

    if comprobar_user_movie_exist:
       #Obtener indice de la pelicula dada
        index=obtener_index_movieId(dicc_user_movies, movie_id)

        # Obtén la lista de similitudes del coseno para una película y usuario dada
        cosine_sim_list = simil_cosine.iloc[index]
        
        # Ordena las películas en función de la similitud del coseno
        most_similar_movies_sorted=cosine_sim_list.sort_values(ascending=False)

        # Obtiene los índices de las 5 películas más similares superior al umbral 
        most_similar_movies = most_similar_movies_sorted[most_similar_movies_sorted > threshold][:5].index

        # Obtiene los títulos de las películas más similares
        most_similar_movie_titles = list(set([dicc_user_movies['title'][id] for id in most_similar_movies if id in dicc_user_movies['title']]))
        
        # Obtener los titulos de todas las peliculas vistas por el usuario
        titles_saws = get_title_from_movieId(user_id ,dicc_user_movies)
        titles_saws = list(titles_saws)
        
        # Pelicula actual pasada por parametros (title)
        actual_title = dicc_user_movies['movieId'].loc[dicc_user_movies['movieId'] == movie_id].index[0]

        print('Porque has visto:',dicc_user_movies['title'].loc[actual_title])
        
        # Filtra las películas recomendadas para excluir las que el usuario ya ha visto
        recommended_movies = [movie for movie in most_similar_movie_titles if movie not in titles_saws]

        # Si las peliculas se devuelven vacias se retorna la condicion
        if not recommended_movies:
            return 'No se encontro una pelicula lo suficientemente significativa para recomendar'
        print('Recomendamos:')
        return recommended_movies
    else: 
        return 'No se encontro usuario para pelicula dada'
        

In [25]:
result_recommendation=get_recommendations(136540,36)#userId, movieId
result_recommendation

Porque has visto: Dead Man Walking
Recomendamos:


['Casino', 'Negotiator, The']

Podemos ver que para la pelicula Dead Man Walking (cuyo genero es crimen/suspenso lanzada en el 1995), 

se recomiendan dos peliculas de crimenes y suspense, ambas lanzadas en el 1995 y 1998.

In [16]:
list_user_movies.tail(10)

,userId,movieId,rating,title
19991,92360.0,4816,3.0,Zoolander
19992,35192.0,2012,3.0,Back to the Future Part III
19993,91709.0,1499,2.5,Anaconda
19994,22337.0,1090,3.5,Platoon
19995,136540.0,36,3.0,Dead Man Walking
19996,44535.0,38388,2.5,Goal! The Dream Begins
19997,136429.0,315,4.0,"Specialist, The"
19998,39514.0,354,3.0,Cobb
19999,106343.0,377,5.0,Speed
20000,138459.0,39444,4.0,"Weather Man, The"


In [17]:
# Buscar usuarios 
def movies_peruser(idUser):    
    print(list_user_movies[list_user_movies['userId']==idUser])

In [24]:
movies_peruser(136540)

         userId  movieId  rating             title
19995  136540.0       36     3.0  Dead Man Walking
